<a href="https://colab.research.google.com/github/emidiob/ML_Art_Resources/blob/main/MidasV2_Tensorflow_ImageSequence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**If you find these notebooks helpful, support by following me on [Instagram](https://www.instagram.com/emidiobattipaglia/) and [Twitter](https://twitter.com/Emidio_B)**

# ⭕ Check Hardware

In [ ]:
print('CPU:') 
!lscpu |grep 'Model name'
print('GPU:') 
!nvidia-smi -q
!nvidia-smi
print('Memory:') 
!free -h --si | awk  '/Mem:/{print $2}'
print('Memory Info:') 
!cat /proc/meminfo

# ⭕ Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#⭕ Move the archive in the main directory and unzip files


In [ ]:
!cp /content/drive/MyDrive/midas/input.zip /content/

!pip install patool
import patoolib
patoolib.extract_archive("/content/input.zip",outdir='/content/')

!rm /content/input.zip



#⭕ Install Dependencies

In [ ]:
pip install timm tensorflow tensorflow-hub opencv-python matplotlib


# ⭕ Import libraries and create Export folder

In [ ]:
import cv2
import numpy as np
import urllib.request
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import os

path = "/content/export"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)



#⭕ Analize and export images

In [ ]:
# the runtime initialization will not allocate all memory on the device to avoid out of GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    #tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.experimental.set_virtual_device_configuration(gpu,
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])

# load model
module = hub.load("https://tfhub.dev/intel/midas/v2/2", tags=['serve'])

directory = r'/content/input'
for entry in os.scandir(directory):
    if (entry.path.endswith(".jpg")
            or entry.path.endswith(".png")
                    or entry.path.endswith(".jpeg")) and entry.is_file():
        print("Analizzo l'immagine", entry.name)
        #filename = entry.path
        # input
        img = cv2.imread(entry.path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) / 255.0

        img_resized = tf.image.resize(img, [384,384], method='bicubic', preserve_aspect_ratio=False, align_corners=True)
        img_resized = tf.transpose(img_resized, [2, 0, 1])
        img_input = img_resized.numpy()
        reshape_img = img_input.reshape(1,3,384,384)
        tensor = tf.convert_to_tensor(reshape_img, dtype=tf.float32)


        output = module.signatures['serving_default'](tensor)
        prediction = output['default'].numpy()
        prediction = prediction.reshape(384, 384)
                    
        # output file
        prediction = cv2.resize(prediction, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_CUBIC)
        print(" Write image to:", entry.name)
        depth_min = prediction.min()
        depth_max = prediction.max()
        img_out = (255 * (prediction - depth_min) / (depth_max - depth_min)).astype("uint8")

        #cv2.imwrite(entry.name, img_out)

        #path = '/content/export'
        #cv2.imwrite(os.path.join(path , entry.name), img)
        cv2.imwrite(f'/content/export/{entry.name}',img_out)
        #plt.imshow(img_out)
        # plt.show()


!zip -r /content/export.zip /content/export

!cp /content/export.zip /content/drive/MyDrive/midas/